#  EDI Parser and Generator

In [4]:
from collections import defaultdict


In [5]:
class edi_parser:
    def __init__(self, file_path):
        self.file_path = file_path
        self.segments = []
        with open(file_path, "r") as f:
            data = f.read()
            self.segments = data.split("~")

        self.result_dict = defaultdict(list)
        
    
    def parse_834(self):
        
        i = 0 
        for segment in self.segments:
            segment_data = segment.split('*')
            key = segment_data[0]
            i += 1 #i is the positional value of the data segment
            values = [value for value in segment_data[1:]]
            values.append(i) # the position is plased at the end of the values list 
            self.result_dict[key].append(values)
            
        return self.result_dict
    
    def ins_data(self):
        
        for key, values in self.result_dict.items():
            if key == "INS":
                return values
            
    def nm1_data(self):
        
        for key, values in self.result_dict.items():
            if key == "NM1":
                return values
    
class edi_generator:
    
    def __init__(self,res_dict):
        self.res_dict = res_dict
    
    def generate_834(self, w_file_path):
        
        segments = []
        for key, values in self.res_dict.items():
            for value_set in values:
                segment_data = [key] + value_set[:]
                segment = '*'.join(map(str, segment_data))
                segments.append(segment)

        # Sort the segments based on the position
        segments.sort(key=lambda x: int(x.split('*')[-1]))

        #Remove the positional element from the joined segments
        segments = [segment.rsplit('*', 1)[0] for segment in segments]

        #Join the segments with '~' delimiter
        data_834 = '~'.join(segments)
        #print(data)
        # Write the data to the file
        with open(w_file_path, "w") as file:
            file.write(data_834)
        
        return data_834

### Parsing the File

In [6]:
file_path = "MCE834Sample_3.txt"
file2 = edi_parser(file_path)
parsed_data = file2.parse_834()

print(parsed_data)

defaultdict(<class 'list'>, {'ISA': [['00', '          ', '00', '          ', 'ZZ', 'EMEDNYMCR      ', 'ZZ', '8-DIGIT PLAN ID', '191127', '1410', '^', '00501', '193310003', '0', 'T', ':', 1]], 'GS': [['BE', 'EMEDNYMCR', 'ETIN', '20191127', '141000', '193310003', 'X', '005010X220A1', 2]], 'ST': [['834', '193310001', '005010X220A1', 3], ['834', '193310002', '005010X220A1', 79], ['834', '193310003', '005010X220A1', 148], ['834', '193310004', '005010X220A1', 237]], 'BGN': [['00', '1933100000000003XF1933100000000001', '20191127', '141001', '', '', '', '2', 4], ['00', '1933100000000003XF1933100000000002', '20191127', '141002', '', '', '', '2', 80], ['00', '1933100000000003XF1933100000000003', '20191127', '141003', '', '', '', '2', 149], ['00', '1933100000000003XF1933100000000004', '20191127', '141004', '', '', '', '2', 238]], 'QTY': [['TO', '1', 5], ['TO', '1', 81], ['TO', '1', 150], ['TO', '1', 239]], 'N1': [['P5', 'MEDICAID', 'FI', '141797357', 6], ['IN', '', '94', '8-DIGIT PLAN ID', 7], [

## Generating the 834 text file

In [7]:
w_file_path = "rev_MCE834Sample_3.txt"
file_gen = edi_generator(parsed_data)
result_834 = file_gen.generate_834(w_file_path)

print(result_834)

ISA*00*          *00*          *ZZ*EMEDNYMCR      *ZZ*8-DIGIT PLAN ID*191127*1410*^*00501*193310003*0*T*:~GS*BE*EMEDNYMCR*ETIN*20191127*141000*193310003*X*005010X220A1~ST*834*193310001*005010X220A1~BGN*00*1933100000000003XF1933100000000001*20191127*141001****2~QTY*TO*1~N1*P5*MEDICAID*FI*141797357~N1*IN**94*8-DIGIT PLAN ID~INS*Y*18*021*28*A***AC~REF*0F*XX99992X~REF*17*XX99992X~REF*3H*9929999999~REF*ABB*XX99992X~DTP*356*D8*20190101~NM1*IL*1*SUBSCRIBER 2 LAST NAME*SUBSCRIBER 2 FIRST NAME*MI***34*999999992~PER*IP**TE*9999992222~N3*122 BEE ST*CARE OF NAME FOR SUB 2~N4*BNYTOWN*NY*14001**CY*36029~DMG*D8*20020202*F**C~LUI*LE*FRE**6~LUI*LE*FRE**7~NM1*31*1~N3*CARE OF MAILING ADDRESS FOR SUB 2*APT BB~N4*BNYTOWN*NY*14001~HD*021**HLT**IND~DTP*348*D8*20190101~LX*1~NM1*Y2*2******SV*8-DIGIT PLAN ID*72~LS*2700~LX*1~N1*75*FAM IND~REF*17*I~LX*2~N1*75*COE CODE~REF*17*30~DTP*007*D8*20190101~LX*3~N1*75*MONEY CODE~REF*17*00~DTP*007*D8*20190101~LX*4~N1*75*FISCAL COUNTY~REF*17*36029~DTP*007*D8*20190101~LX*5~N1

## Getting Info 

In [8]:
class NM1():
        
    def __init__(self, data, loop):
        
        self.data = data[loop] 
        
        self.entityIdentifierCode = None
        self.entityTypeQualifier = None
        self.lastName = None
        self.firstName = None
        self.middleName = None
        self.suffix = None
        self.idCodeQualifier = None
        self.idCode = None
        
    
    #to-do implement such that if the data is given then set new values else keep the previous values as default
    def set_values(self, elements = None):
        
        if elements == None:
            elements = self.data

        self.entityIdentifierCode = elements[0]
        self.entityTypeQualifier = elements[1]
        self.lastName = elements[2]
        self.firstName = elements[3]
        self.middleName = elements[4]
        self.suffix = elements[5]
        self.idCodeQualifier = elements[6]
        self.idCode = elements[7]
#         self.position = element[]
            
    def get_values(self):
#         for sublist in self.data:
#             return sublist
        print('entityIdentifierCode :',self.entityIdentifierCode)
        print('entityTypeQualifier :',self.entityTypeQualifier)  
        print('lastName :',self.lastName)  
        print('firstName :',self.firstName)  
        print('middleName :',self.middleName) 
        print('suffix :',self.suffix)  
        print('idCodeQualifier :',self.idCodeQualifier)
        print('idCode :',self.idCode)
#         print(self.position)  
        


In [10]:
nm1_file_path = "/Users/rupakpantha/Downloads/viveka834/MCE834Sample_3.txt"

file_p = edi_parser(nm1_file_path)
parsed_data = file_p.parse_834()

nm1_data = file_p.nm1_data()

#print(nm1_data)

# element = []
#print(nm1_data[1])
# for i in range(len(ins_data[3])):
#     element[i] = ins_data[3][i]

# print(element)

file834_nm1_0 = NM1(nm1_data,0)
file834_nm1_0.set_values()
file834_nm1_0.get_values()

entityIdentifierCode : IL
entityTypeQualifier : 1
lastName : SUBSCRIBER 2 LAST NAME
firstName : SUBSCRIBER 2 FIRST NAME
middleName : MI
suffix : 
idCodeQualifier : 
idCode : 34
